In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [17]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)

In [18]:
salary_file = pd.ExcelFile('contract_data/salaries_1985to2020_final.xlsx')
salaries = pd.read_excel(salary_file, 0)
players_info = pd.read_excel(salary_file, 1)

In [19]:
### add years of the contract
years = list(salaries.groupby('player_id').apply(lambda x: [i for i in range(1, len(x)+1)]))
flatten = lambda l: [item for sublist in l for item in sublist]
salaries['years'] = flatten(years)

In [20]:
salaries_98_20 = salaries.loc[salaries['season_end'] >= 1999]
unique_ids = list(pd.unique(salaries_98_20['player_id']))
filtered_players_info = players_info.loc[[True if pid in unique_ids else False for pid in players_info['_id']]]

In [21]:
# Deal with rookie contracts of first round picks and second round picks
# separately. 
first_round_picks = filtered_players_info.loc[filtered_players_info['draft_round'] == '1st round']
second_round_picks = filtered_players_info.loc[filtered_players_info['draft_round'] == '2nd round']

In [22]:
salaries_first_round = salaries_98_20.loc[[True if pid in list(first_round_picks['_id']) else False for pid in salaries_98_20['player_id']]].drop(columns=['team'])
# Drop entries with 'years' being 4 or less
# Prior to 2011 CBA, first round rookie contract is a 3+1 contract with the fourth year being team option
# After 2011 CBA, first round rookie contract is a 2+2 contract with both the 3rd and 4th year being team options
# Dropping values will potentially over-dropping some datapoints
salaries_first_round = salaries_first_round.loc[salaries_first_round['years'] > 4]

salaries_second_round = salaries_98_20.loc[[True if pid in list(second_round_picks['_id']) else False for pid in salaries_98_20['player_id']]].drop(columns=['team'])
# Drop entries with 'years' being 3 or less
salaries_second_round = salaries_second_round.loc[salaries_second_round['years'] > 3]

In [23]:
# Following part is for fixing data in all stats

In [24]:
stats = pd.read_excel('season_data/advanced_stats/FINAL_ADVANCED_STATS/all_seasons_advanced_stats.xlsx')

In [25]:
stats['error_in_name'] = stats['Year'].apply(lambda x: x is np.nan)
stats_need_fix = stats.loc[stats['error_in_name'] == True]

In [26]:
stats_need_fix['Year'] = '2019-20'
name = stats_need_fix['name'].apply(lambda x: re.sub(r"\\.+", "", x))
slug = stats_need_fix['name'].apply(lambda x: re.sub(r".+\\", "", x))
stats_need_fix['name'], stats_need_fix['slug'] = name, slug

<ipython-input-26-5b3e32999086>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_need_fix['Year'] = '2019-20'
<ipython-input-26-5b3e32999086>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_need_fix['name'], stats_need_fix['slug'] = name, slug


In [27]:
stats = stats.loc[stats['error_in_name'] == False].append(stats_need_fix).drop(columns=['error_in_name'])

In [28]:
# Create a primary key in two salaries table for joins
salaries_first_round['key'] = salaries_first_round['player_id'] + " " + salaries_first_round['season']
salaries_second_round['key'] = salaries_second_round['player_id'] + " " + salaries_second_round['season']
stats['key'] = stats['slug'] + " " + stats['Year']

In [15]:
stats.loc[stats['key'] == 'zellety01 2017-18']

,name,positions,age,team,games_played,minutes_played,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,block_percentage,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,is_combined_totals,Year,slug,key
16534,Tyler Zeller,CENTER,28,BROOKLYN NETS,42,703,15.3,0.587,0.114,0.262,9.4,20.0,14.7,6.5,0.6,2.2,12.1,17.9,1.0,0.6,1.5,0.105,-0.7,-0.4,-1.0,0.2,0.0,2017-18,zellety01,zellety01 2017-18
16535,Tyler Zeller,CENTER,28,MILWAUKEE BUCKS,24,406,17.1,0.622,0.019,0.181,13.6,18.4,16.0,7.0,0.9,2.9,9.6,13.9,1.1,0.3,1.4,0.163,-0.3,-1.8,-2.1,0.0,0.0,2017-18,zellety01,zellety01 2017-18


In [29]:
salaries_first_round.join(stats.set_index('key'), on='key', how='left')

,names,player_id,salary,season,season_end,season_start,years,key,name,positions,age,team,games_played,minutes_played,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,block_percentage,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,is_combined_totals,Year,slug
17,Mahmoud Abdul-Rauf,abdulma02,798500,2000-01,2001,2000,9,abdulma02 2000-01,Mahmoud Abdul-Rauf,POINT GUARD,31.0,VANCOUVER GRIZZLIES,41.0,486.0,16.7,0.514,0.057,0.118,1.2,4.7,2.9,32.7,1.0,0.1,9.1,26.8,0.7,0.0,0.7,0.070,1.1,-2.1,-1.0,0.1,0.0,2000-01,abdulma02
22,Tariq Abdul-Wahad,abdulta01,5062500,2001-02,2002,2001,5,abdulta01 2001-02,Tariq Abdul-Wahad,SHOOTING GUARD,27.0,DENVER NUGGETS,20.0,417.0,10.3,0.424,0.014,0.221,10.2,11.2,10.7,8.7,2.3,1.6,13.1,19.7,-0.2,0.3,0.0,0.005,-3.1,-0.5,-3.6,-0.2,0.0,2001-02,abdulta01
22,Tariq Abdul-Wahad,abdulta01,5062500,2001-02,2002,2001,5,abdulta01 2001-02,Tariq Abdul-Wahad,SHOOTING GUARD,27.0,DALLAS MAVERICKS,4.0,24.0,0.3,0.000,0.000,0.500,9.4,18.2,13.9,10.4,4.3,3.0,55.1,10.2,-0.1,0.0,-0.1,-0.128,-9.0,2.7,-6.3,0.0,0.0,2001-02,abdulta01
23,Tariq Abdul-Wahad,abdulta01,5625000,2002-03,2003,2002,6,abdulta01 2002-03,Tariq Abdul-Wahad,SHOOTING GUARD,28.0,DALLAS MAVERICKS,14.0,204.0,12.4,0.470,0.017,0.103,7.6,14.0,10.8,15.4,1.5,1.1,10.3,15.0,0.2,0.2,0.4,0.104,-1.6,0.2,-1.4,0.0,0.0,2002-03,abdulta01
24,Tariq Abdul-Wahad,abdulta01,6187500,2003-04,2004,2003,7,abdulta01 2003-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,Tyler Zeller,zellety01,1709538,2017-18,2018,2017,6,zellety01 2017-18,Tyler Zeller,CENTER,28.0,BROOKLYN NETS,42.0,703.0,15.3,0.587,0.114,0.262,9.4,20.0,14.7,6.5,0.6,2.2,12.1,17.9,1.0,0.6,1.5,0.105,-0.7,-0.4,-1.0,0.2,0.0,2017-18,zellety01
15170,Tyler Zeller,zellety01,1709538,2017-18,2018,2017,6,zellety01 2017-18,Tyler Zeller,CENTER,28.0,MILWAUKEE BUCKS,24.0,406.0,17.1,0.622,0.019,0.181,13.6,18.4,16.0,7.0,0.9,2.9,9.6,13.9,1.1,0.3,1.4,0.163,-0.3,-1.8,-2.1,0.0,0.0,2017-18,zellety01
15171,Tyler Zeller,zellety01,1933941,2018-19,2019,2018,7,zellety01 2018-19,Tyler Zeller,CENTER,29.0,ATLANTA HAWKS,2.0,11.0,1.5,0.000,0.500,0.000,18.7,39.1,28.7,10.6,0.0,0.0,0.0,7.4,0.0,0.0,0.0,-0.013,-8.7,-1.6,-10.2,0.0,0.0,2018-19,zellety01
15171,Tyler Zeller,zellety01,1933941,2018-19,2019,2018,7,zellety01 2018-19,Tyler Zeller,CENTER,29.0,MEMPHIS GRIZZLIES,4.0,82.0,19.4,0.640,0.000,0.643,12.1,12.5,12.3,6.2,0.6,3.5,10.0,21.8,0.2,0.1,0.3,0.191,-2.3,-1.9,-4.3,0.0,0.0,2018-19,zellety01


In [ ]:
def standardize(df, column):
    series = []
    for i in df[column]:
        series.append((i - df[column].mean())/df[column].std())
    return series

def correlation(df, column_1, column_2):
    return np.mean(df[column_1] * df[column_2])

def scatter_and_correlation(df, column_1, column_2):
    df[column_1 + "_standard"] = standardize(df, column_1)
    df[column_2 + "_standard"] = standardize(df, column_2)
    df.plot.scatter(x = column_1 + "_standard", y = column_2 + '_standard')
    return correlation(df, column_1 + '_standard', column_2 + '_standard')